In [1]:
import os
import sys
import requests
import re
import time
import json
import pandas as pd
import csv
from datetime import datetime
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By

import warnings
warnings.filterwarnings('ignore')

In [2]:
extracts = re.compile('[^ 가-힣|a-z|A-Z|0-9|\[|\]|(|)|-|~|?|!|.|,|:|;|%]+')
cost_extracts = re.compile('[^ 가-힣|a-z|A-Z|\[|\]|(|)|-|~|?|!|.|,|:|;|%]+')

In [3]:
data = open('./data/goodchoice.csv', 'w', encoding='utf8')

In [4]:
data.write('"basic_info"')
data.write(', ')
data.write('"cost_info"')
data.write('\n')

1

In [5]:
driver = webdriver.Chrome("chromedriver.exe")
driver.maximize_window() #전체 화면

In [6]:
driver.get('https://www.goodchoice.kr/')
time.sleep(2)

In [7]:
# 서울 강남/역삼/논현 검색
driver.find_element(By.XPATH, "/html/body/div[1]/div[6]/div[2]/ul/li[1]/a").click()
driver.find_element(By.XPATH, "/html/body/div[1]/div[6]/div[2]/div/div/div").click()
driver.find_element(By.XPATH, "/html/body/div[1]/div[6]/div[3]/div[1]/div[2]/div[1]/div/ul/li[1]/a").click()
driver.find_element(By.XPATH, "/html/body/div[1]/div[6]/div[3]/div[1]/div[2]/div[2]/div/ul[1]/li[2]/a").click()
driver.find_element(By.XPATH, "/html/body/div[1]/form/div/div[3]/div[3]/div[2]/li[1]").click()

In [8]:
# # 리뷰 크롤링
# for i in range(1,10):
#     try:
#         driver.find_element(By.XPATH, "/html/body/div[1]/form/div/div[3]/div[3]/div[2]/li[{}]".format(i)).click()
#         driver.find_element(By.XPATH, "/html/body/div[1]/div[6]/div[2]/button[3]").click()
#         driver.back()
#     except:
#         driver.close()
#         continue

In [9]:
# 기본 정보 및 요금 정보
rs = requests.get('https://www.goodchoice.kr/product/detail?ano=2084&adcno=1')
rs.encoding = None
html = rs.text
soup = bs(html, 'html.parser')

tarbasic_info = soup.select('#content > article.detail_info > section.default_info')
tarcost_info = soup.select('#content > article.detail_info.on > section.table_wrap > div:nth-child(2) > table:nth-child(2)')

basic_info = ''
for text in tarbasic_info:
    basic_info += text.getText()

basic_info = extracts.sub('', basic_info)
basic_info = re.sub(' +', ' ', basic_info)
basic_info = '"'+basic_info+'"'

rcost_info = soup.find("table", {"class" : "table_type_02"})
cost_info = rcost_info.find("span")
cost_info = cost_info.text

data.write(basic_info)
data.write('\n')
data.write(cost_info)
data.write('\n')

data.close()